In [ ]:
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as KL
from keras import callbacks
from matplotlib import pyplot as plt
import cv2
import math
import os

In [ ]:
path = os.getcwd()
dir_list = os.listdir(path)

In [ ]:
# Create labels
# img_0 - img_118 are Treble. Treble = 0
# img_119 - img_219 are Bass. Bass = 1
# img_220 - img_318 are Tenor. Tenor = 2
# img_319 - img_418 are Alto. Alto = 3

labels = np.empty(419,dtype=int) # Change number to 220 for Model 1. 200 for Model 2. 419 for Model 3.
for x in range(119):
  labels[x] = 0
for x in range(119,220):
  labels[x] = 1

#####
# Comment out this part for Model 1
for x in range(220,319):
  labels[x] = 2
for x in range(319,419):
  labels[x] = 3
#####


#####
# This part is for Model 2
#for x in range(100):
#    labels[x] = 2
#for x in range(100,200):
#    labels[x] = 3
#####

In [ ]:
imgpath = [f'img_{n}.png' for n in range(419)] # 220 for Model 1. 200 for Model 2. 419 for Model 3.

imgs = []
maxh = 0
maxw = 0
for ipath in imgpath:
  # Read in image in greyscale
  fullpath = os.path.join(path,ipath)
  img = cv2.imread(fullpath,0)

  # Image processing
  size = np.shape(img)
  
  if (size[0]>maxh):
    maxh = size[0]
  if (size[1]>maxw):
    maxw = size[1]

  img_new = np.zeros(shape = size)

  img_new = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,61,21)

  # Gaussian blur for noise reduction
  img_gaus = cv2.GaussianBlur(img_new,(5,5),0)
  img_gaus = np.uint8(img_gaus)

  # Resize and pad so all photos are the same size
  h = size[0]
  w = size[1]
  if (h>w):
    scale = 7015/h
    new_w = int(size[1] * scale)
    padding = 7015 - new_w
    padding_floor = math.floor(padding/2)
    padding_ceil = math.ceil(padding/2)
    resized = cv2.resize(img_gaus, (new_w,7015), interpolation = cv2.INTER_AREA)
    img_gaus = cv2.copyMakeBorder(resized, 0, 0, padding_floor, padding_ceil, cv2.BORDER_CONSTANT, None, value = 0)
    img_gaus = cv2.resize(img_gaus, (512,512), interpolation = cv2.INTER_AREA)
  elif (w>h):
    scale = 7015/w
    new_h = int(size[0] * scale)
    padding = 7015 - new_h
    padding_floor = math.floor(padding/2)
    padding_ceil = math.ceil(padding/2)
    resized = cv2.resize(img_gaus, (7015,new_h), interpolation = cv2.INTER_AREA)
    img_gaus = cv2.copyMakeBorder(resized, padding_floor, padding_ceil, 0, 0, cv2.BORDER_CONSTANT, None, value = 0)
    img_gaus = cv2.resize(img_gaus, (512,512), interpolation = cv2.INTER_AREA)
  else:
    img_gaus = cv2.resize(img_gaus, (512,512), interpolation = cv2.INTER_AREA)

  img_gaus = np.array(img_gaus)
  img_gaus = img_gaus.astype('float32')

  imgs.append(img_gaus)

print("Done")

In [ ]:
#####
# Models 1 and 3
Xtrain = np.concatenate((imgs[0:96],imgs[119:200],imgs[220:300],imgs[319:400]),axis=0,dtype='float32') 
# ^^^ Change to imgs[0:96],imgs[119:200] only for Model 1
ytrain = np.concatenate((labels[0:96],labels[119:200],labels[220:300],labels[319:400]),axis=0,dtype=int) 
# ^^^ Change to labels[0:96],labels[119:200] only for Model 1
Xtest = np.concatenate((imgs[96:119],imgs[200:220],imgs[300:319],imgs[400:419]),axis=0,dtype='float32') 
# ^^^ Change to imgs[96:119],imgs[200:220] only for Model 1
ytest = np.concatenate((labels[96:119],labels[200:220],labels[300:319],labels[400:419]),axis=0,dtype=int) 
# ^^^ Change to ((labels[96:119],labels[200:220] only for Model 1
#####

#####
# Uncomment this for Model 2
#Xtrain = np.concatenate((imgs[0:80],imgs[100:181]),axis=0,dtype='float32') #imgs[0:96],imgs[119:200],imgs[220:300],imgs[319:400]
#ytrain = np.concatenate((labels[0:80],labels[100:181]),axis=0,dtype=int) #labels[0:96],labels[119:200],labels[220:300],labels[319:400]
#Xtest = np.concatenate((imgs[80:100],imgs[181:200]),axis=0,dtype='float32') #imgs[96:119],imgs[200:220],imgs[300:319],imgs[400:419]
#ytest = np.concatenate((labels[80:100],labels[181:200]),axis=0,dtype=int) #labels[96:119],labels[200:220],labels[300:319],labels[400:419]
#####

In [ ]:
# Making the layers
inputs = KL.Input(shape=(512, 512))
l = KL.Flatten()(inputs) 
# Dense Layers: https://keras.io/api/layers/core_layers/dense/   
l = KL.Dense(512, activation=tf.nn.relu)(l) # Relu activation function
outputs = KL.Dense(10, activation=tf.nn.softmax)(l) # softmax activation function

model = tf.keras.models.Model(inputs, outputs)
model.summary()

#earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode = "min", patience = 5, restore_best_weights = True)

# For compile/fit args: https://www.tensorflow.org/api_docs/python/tf/keras/Model
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"]) # sparse bc labels as ints
history = model.fit(Xtrain, ytrain, epochs=100, validation_data=(Xtest, ytest))

In [ ]:
#  For history and plotting https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Test'], loc='upper left')
plt.show()